In [ ]:
import math
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from numpy import sin, cos, arccos, pi, round

Lendo a CSV e removendo todos os dados com valores de Tp_est iguais a zero

In [ ]:
df_raw_data = pd.read_csv('data.csv')
df_raw_data.drop(df_raw_data[df_raw_data['Tp_est'] == 0.0].index, inplace=True);

Remoção inicial de colunas da base geral (2018 a 2022). A colunas removidas foram:
- Unnamed: 0 pois é a coluna de ids
- latitude e longitude pois possuem a mesma informação que as colunas lat e lon;
- distancia: Não agrega valor ao modelo.

In [ ]:
df_raw_data.drop(['Unnamed: 0', 'latitude', 'longitude', 'distancia'], axis=1, inplace=True)

Separando o raw data entre treinamento e teste. Anos de 2018 a 2021 para treinamento e 2022 para teste.

In [ ]:
raw_train_test_group = df_raw_data.groupby(df_raw_data['time'].str.contains('2022'))

df_raw_train = raw_train_test_group.get_group(False).copy()
df_raw_test = raw_train_test_group.get_group(True).copy()

Remoção das colunas elevation e sweep pois não possuiam valor agregado na base

Para verificar que as colunas elevation e sweep não possuiam valor agregado, foi utilizado a função describe do pandas. Com esta função, foi possível verificar que ambas possuiam média, minimo e maximo identicos, além de um desvio padrão igual a 0, ou seja, todas as linhas possuiam o mesmo valor.

In [ ]:
print(df_raw_train.describe()['elevation'], end="\n\n")
print(df_raw_train.describe()['sweep'])

df_raw_train.drop(['elevation','sweep'], axis=1, inplace=True)

Criando 2 DataFrames onde um deles armazena as informações das colunas X e Y da base e o outro armazena as colunas LAT e LON

In [ ]:
grouped_by_est = df_raw_train.groupby(['Est'])

df_xy = pd.DataFrame(columns=['est','x', 'y'])
df_lat_lon = pd.DataFrame(columns=['est', 'lat', 'lon'])
for est in grouped_by_est.groups.keys():
    group = grouped_by_est.get_group(est)

    list_xy = [est, group['x'].mean(), group['y'].mean()]
    df_xy = pd.concat([pd.DataFrame([list_xy], columns=df_xy.columns), df_xy], ignore_index=True)

    list_lat_lon = [est, group['lat'].mean(), group['lon'].mean()]
    df_lat_lon = pd.concat([pd.DataFrame([list_lat_lon], columns=df_lat_lon.columns), df_lat_lon], ignore_index=True)

Verificando que os valores X e Y fazer o mesmo papel que os valores de LAT e LON para o calcula da distância entre as estações

In [ ]:
def rad2deg(radians):
    degrees = radians * 180 / pi
    return degrees

def deg2rad(degrees):
    radians = degrees * pi / 180
    return radians

theta = df_lat_lon['lon'][0] - df_lat_lon['lon'][1]
distance = 60 * 1.1515 * rad2deg(
    arccos(
        (sin(deg2rad(df_lat_lon['lat'][0])) * sin(deg2rad(df_lat_lon['lat'][1]))) + 
        (cos(deg2rad(df_lat_lon['lat'][0])) * cos(deg2rad(df_lat_lon['lat'][1])) * cos(deg2rad(theta)))
    )
)
distance = round(distance * 1.609344, 5)
print(distance)

# Distância euclidina por meio do numpyu
point1 = np.array((df_xy['x'][0], df_xy['y'][0]))
point2 = np.array((df_xy['x'][1], df_xy['y'][1]))
distance = np.linalg.norm(point1 - point2)
print(round(distance / 1000, 5))


Plotando um gráfico das posições das estações para ter um auxilio visual na clusterização

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
for data in df_xy.iterrows():
    ax.scatter(data[1][1], data[1][2], label=f'{data[1][0]} - {data[0]}')

colormap = plt.cm.gist_ncar
colorst = [colormap(i) for i in np.linspace(0.1, 0.9,len(ax.collections))]
for t, j1 in enumerate(ax.collections):
    j1.set_color(colorst[t])

for data in df_xy.iterrows():
    ax.annotate(data[0], (data[1][1], data[1][2]))

ax.legend(bbox_to_anchor=(1, 1), bbox_transform=ax.transAxes, fontsize='small')
plt.show()

Calculando a distancia de cada estação entre si para ter um embasamento numérico na decisão da clusterização

In [ ]:
def euclidian_distance(x1, y1, x2, y2):
    p1 = np.array((x1, y1))
    p2 = np.array((x2, y2))
    return np.linalg.norm(p1 - p2)

est_distances = []
for est in df_xy.iterrows():
    distances = []
    for row in df_xy.iterrows():
        if est[1][0] != row[1][0]:
            distance = round((euclidian_distance(est[1][1], est[1][2], row[1][1], row[1][2]) / 1000), 3)
            distances.append((distance, row[1][0]))
    distances = sorted(distances)
    distances_sum = sum(i[0] for i in distances[:6])
    est_distances.append((distances_sum, est[1][0], distances))

est_distances = sorted(est_distances)

for est in est_distances:
    print(f'Estação: {est[1]} - Soma: {est[0]} - Estações: {est[2][:6]}')

Remoção das colunas X, Y, Z, LAT, LON, ALT, ja que a partir de agora, a clusterização vai trazer a informação de posicionamento

In [ ]:
df_raw_train.drop(['x', 'y', 'z', 'lat', 'lon', 'alt'], axis=1, inplace=True)

Após alguns testes e tentativas de divisão, a clusterização final ficou como:
- Grupo 1: Laranjeiras_do_Sul, Segredo, Derivacao_do_Rio_Jordao, Coronel_Domingos_Soares, Solais_Novo
- Grupo 2: Pato_Branco, Bela_Vista_Jusante, Aguas_do_Vere
- Grupo 3: Baixo_Iguacu, Salto_Caxias, Reservatorio_Salto_Caxias, Boa_Vista_da_Aparecida, Porto_Santo_Antonio
- Grupo 4: Santa_Helena, Foz_do_Iguacu_-_Itaipu
- Grupo 5: Toledo, Cascavel, Assis_Chateaubriand
- Grupo 6: Guaira, Palotina, Altonia
- Grupo 7: Umuarama, Ubirata, Porto_Formosa
- Grupo 8: Loanda, Paranavai, Campo_Mourao

In [ ]:
grouped_by_est = df_raw_train.groupby(['Est'])
dict_df_train = {'df_train' : {}, 'for_input': {}}

df_groupped = grouped_by_est.get_group('Laranjeiras_do_Sul')
df_groupped = pd.concat([grouped_by_est.get_group('Segredo'), df_groupped], ignore_index=True)
df_groupped = pd.concat([grouped_by_est.get_group('Derivacao_do_Rio_Jordao'), df_groupped], ignore_index=True)
df_groupped = pd.concat([grouped_by_est.get_group('Coronel_Domingos_Soares'), df_groupped], ignore_index=True)
df_groupped = pd.concat([grouped_by_est.get_group('Solais_Novo'), df_groupped], ignore_index=True)
dict_df_train['df_train']['Laranjeiras_do_Sul - Segredo - Derivacao_do_Rio_Jordao - Coronel_Domingos_Soares - Solais_Novo'] = df_groupped

df_groupped = grouped_by_est.get_group('Pato_Branco')
df_groupped = pd.concat([grouped_by_est.get_group('Bela_Vista_Jusante'), df_groupped], ignore_index=True)
df_groupped = pd.concat([grouped_by_est.get_group('Aguas_do_Vere'), df_groupped], ignore_index=True)
dict_df_train['df_train']['Pato_Branco - Bela_Vista_Jusante - Aguas_do_Vere'] = df_groupped

df_groupped = grouped_by_est.get_group('Baixo_Iguacu')
df_groupped = pd.concat([grouped_by_est.get_group('Salto_Caxias'), df_groupped], ignore_index=True)
df_groupped = pd.concat([grouped_by_est.get_group('Reservatorio_Salto_Caxias'), df_groupped], ignore_index=True)
df_groupped = pd.concat([grouped_by_est.get_group('Boa_Vista_da_Aparecida'), df_groupped], ignore_index=True)
df_groupped = pd.concat([grouped_by_est.get_group('Porto_Santo_Antonio'), df_groupped], ignore_index=True)
dict_df_train['df_train']['Baixo_Iguacu - Salto_Caxias - Reservatorio_Salto_Caxias - Boa_Vista_da_Aparecida - Porto_Santo_Antonio'] = df_groupped

df_groupped = grouped_by_est.get_group('Santa_Helena')
df_groupped = pd.concat([grouped_by_est.get_group('Foz_do_Iguacu_-_Itaipu'), df_groupped], ignore_index=True)
dict_df_train['df_train']['Santa_Helena - Foz_do_Iguacu_-_Itaipu'] = df_groupped

df_groupped = grouped_by_est.get_group('Toledo')
df_groupped = pd.concat([grouped_by_est.get_group('Cascavel'), df_groupped], ignore_index=True)
df_groupped = pd.concat([grouped_by_est.get_group('Assis_Chateaubriand'), df_groupped], ignore_index=True)
dict_df_train['df_train']['Toledo - Cascavel - Assis_Chateaubriand'] = df_groupped

df_groupped = grouped_by_est.get_group('Guaira')
df_groupped = pd.concat([grouped_by_est.get_group('Palotina'), df_groupped], ignore_index=True)
df_groupped = pd.concat([grouped_by_est.get_group('Altonia'), df_groupped], ignore_index=True)
dict_df_train['df_train']['Guaira - Palotina - Altonia'] = df_groupped

df_groupped = grouped_by_est.get_group('Umuarama')
df_groupped = pd.concat([grouped_by_est.get_group('Ubirata'), df_groupped], ignore_index=True)
df_groupped = pd.concat([grouped_by_est.get_group('Porto_Formosa'), df_groupped], ignore_index=True)
dict_df_train['df_train']['Umuarama - Ubirata - Porto_Formosa'] = df_groupped

df_groupped = grouped_by_est.get_group('Loanda')
df_groupped = pd.concat([grouped_by_est.get_group('Paranavai'), df_groupped], ignore_index=True)
df_groupped = pd.concat([grouped_by_est.get_group('Campo_Mourao'), df_groupped], ignore_index=True)
dict_df_train['df_train']['Loanda - Paranavai - Campo_Mourao'] = df_groupped

for group in dict_df_train['df_train']:
    df_groupped = dict_df_train["df_train"][group].dropna()
    columns = list(df_groupped.head())

    dict_df_train['for_input'][group] = {}
    for row in df_groupped.iterrows():
        timestamp = datetime.strptime(row[1]["time"], '%Y-%m-%d %H:%M:%S').timestamp()

        row_info = {}
        for col in columns:
            row_info[col] = row[1][col]

        dict_df_train['for_input'][group][timestamp] = row_info


O dicionário 'for_input' produzido acima possui as seguintes características:
- Dividido em grupos da mesma forma que os datasets de treinamento;
- Possui apenas as linhas que possuiam todos os dados dos sensores dos datasets de treinamento;
- Foi organizado por meio de dicionários internos com a chave sendo o timestamp do dado para facilitar a busca.

Com este dicionário, o data input foi realizado de forma clusterizada, ou seja, on inputs foram realizados apenas com dados do própio grupo

In [ ]:
def get_data(row, col, input_values):
    timestamp = datetime.strptime(row["time"], '%Y-%m-%d %H:%M:%S').timestamp()

    value = None
    min_distance = math.inf
    for i in input_values:
        distance = abs(timestamp - i)
        if (distance < min_distance):
            min_distance = distance
            value = input_values[i][col]
            
    return value

def df_input_data(df_group, input_values):
    columns = list(df_group.head())
    for row in df_group.iterrows():
        for col in columns:
            if str(row[1][col]) == 'nan':
                df_group.loc[row[0], col] = get_data(row[1], col, input_values)

for group in dict_df_train['df_train']:
    df_group = dict_df_train['df_train'][group]
    input_values = dict_df_train['for_input'][group]
    df_input_data(df_group, input_values)

To-do: Fazer o data inputation da base de teste seguindo os valores da base de treinamento, e remover as colunas da base de teste que foram removidas na de treinamento